In [19]:
import pandas as pd
import numpy as np
from datetime import datetime
from nbainjuries import injury
from datetime import datetime, timedelta
import warnings
import duckdb
import os
import requests

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}

print("Current working dir:", cwd)
print("RUN_LOCATION =", RUN_LOCATION)

Current working dir: C:/Users/Rodolfo/Jupyter_files/FantasyBasketball/notebooks
RUN_LOCATION = local


In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

folders = os.listdir('../tables/')
df = pd.DataFrame()
for yr in folders:
    df_temp = pd.read_csv(f"../tables/{yr}/season_gamelogs.csv")
    df_temp.insert(0, 'Season', int(yr))
    df = pd.concat([df, df_temp])
df['Date'] = pd.to_datetime(df.Date)
df = df.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df['STL_BLK'] = df.STL + df.BLK
df['PR'] = df.PTS + df.REB 
df['PA'] = df.PTS + df.AST
df['RA'] = df.REB + df.AST
df['PRA'] = df.PTS + df.REB + df.AST

# # CONTROL DATE TO GO BACK AND RELOAD HISTORICAL DATA
# df = df[(df.Date != now)]

df

Today's date: 2025-11-20


,Season,game_id,Date,Team,Team_type,Opp,Opp_type,Player,Pos,Active,MP,FG,FGA,FG%,TPM,TPA,3P%,FT,FTA,FT%,ORB,DRB,REB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,DD,TD,Fpts,STL_BLK,PR,PA,RA,PRA
0,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Christian Braun,SG,1,19.33,2,5,0.400,0,1,0.000,1,2,0.50,1,2,3,2,0,1,1,1,5,3.9,5,0,0,8.5,1,8,7,5,10
1,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Kentavious Caldwell-Pope,SG,1,36.25,8,12,0.667,2,3,0.667,2,2,1.00,1,1,2,1,3,1,3,5,20,15.2,10,0,0,19.0,4,22,21,3,23
2,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Collin Gillespie,PG,1,0.72,0,0,0.000,0,0,0.000,0,0,0.00,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
3,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Aaron Gordon,PF,1,34.98,7,11,0.636,1,2,0.500,0,0,0.00,2,5,7,5,2,1,0,0,15,19.2,6,0,0,26.0,3,22,20,12,27
4,2023,20231024_LAL_DEN,2023-10-24,DEN,Home,LAL,Away,Justin Holiday,SG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.00,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5858,2025,20251119_CHI_POR,2025-11-19,POR,Home,CHI,Away,Yang Hansen,C,1,7.93,1,4,0.250,0,1,0.000,0,0,0.00,1,1,2,1,0,0,1,1,2,-0.1,-5,0,0,3.0,0,4,3,3,5
5859,2025,20251119_CHI_POR,2025-11-19,POR,Home,CHI,Away,Scoot Henderson,PG,0,0.00,0,0,0.000,0,0,0.000,0,0,0.00,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0
5860,2025,20251119_CHI_POR,2025-11-19,POR,Home,CHI,Away,Javonte Cooke,NaN,1,8.72,2,6,0.333,0,1,0.000,0,0,0.00,1,0,1,1,1,0,2,1,4,0.6,-4,0,0,4.0,1,5,5,2,6
5861,2025,20251119_CHI_POR,2025-11-19,POR,Home,CHI,Away,Caleb Love,NaN,1,18.33,1,12,0.083,0,7,0.000,3,4,0.75,1,2,3,2,1,0,0,1,5,-0.1,-20,0,0,9.5,1,8,7,5,10


In [21]:
%run ./common_utils.ipynb

In [22]:
df_gms = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
df_gms = df_gms[(df_gms.Date == now)]
tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
display(df_gms)

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
222,2025-11-20,7:00p,Los Angeles Clippers,LAC,Orlando Magic,ORL,Kia Center,0,0,0
223,2025-11-20,8:00p,Sacramento Kings,SAC,Memphis Grizzlies,MEM,FedExForum,1,0,0
224,2025-11-20,8:00p,Philadelphia 76ers,PHI,Milwaukee Bucks,MIL,Fiserv Forum,1,0,0
225,2025-11-20,8:00p,Atlanta Hawks,ATL,San Antonio Spurs,SAS,Frost Bank Center,0,0,0


In [23]:
df_teams = pd.read_csv("../src/team_info_xref.csv")

df_inj = injury.get_reportdata(datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]), return_df=True)
df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
df_inj['Player'] = df_inj['Player'].str.strip()
df_inj['Date'] = pd.to_datetime(df_inj['Date'])
df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                         JOIN df_teams ON df_inj.Team = df_teams.Team
                         WHERE Date = '{now}' AND Status = 'Out'""").fetchdf()

df_best_out = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Fpts DESC) as Off_Rk FROM
                              (SELECT Team, Player, Pos, AVG(Fpts) as Fpts FROM df 
                              WHERE Season = 2025 AND Fpts > 0 AND Pos != 'None'
                              GROUP BY Team, Player, Pos)""").fetchdf()
df_best_out = con.execute(f"""SELECT * EXCLUDE(t2.Team, t2.Player) FROM df_best_out t1 
                              JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                              ORDER BY Fpts DESC""").fetchdf()
display(df_best_out)

Validated Injury-Report_2025-11-20_04PM.


,Team,Player,Pos,Fpts,Off_Rk,Date,Status,Reason
0,SAS,Victor Wembanyama,C,37.833333,2,2025-11-20,Out,Injury/Illness - Left Calf; Strain
1,MIL,Giannis Antetokounmpo,PF,35.461538,1,2025-11-20,Out,Injury/Illness - Left Adductor; Strain
2,SAC,Domantas Sabonis,C,25.772727,6,2025-11-20,Out,Injury/Illness - Left Knee; Soreness
3,ORL,Paolo Banchero,PF,25.125000,8,2025-11-20,Out,Injury/Illness - Left Groin; Strain
4,SAS,Stephon Castle,PG,23.115385,12,2025-11-20,Out,Injury/Illness - Left Hip Flexor; Strain
5,PHI,Joel Embiid,C,21.416667,16,2025-11-20,Out,Injury/Illness - Right Knee; Injury Management
6,ATL,Trae Young,PG,19.600000,19,2025-11-20,Out,Injury/Illness - Right Knee; MCL Sprain
7,PHI,Kelly Oubre Jr.,PF,19.318182,15,2025-11-20,Out,Injury/Illness - Left Knee; Sprain
8,MEM,Ja Morant,PG,19.125000,22,2025-11-20,Out,Injury/Illness - Right Calf; Strain
9,SAS,Dylan Harper,SG,14.750000,26,2025-11-20,Out,Injury/Illness - Left Calf; Strain


In [24]:
def get_sportsbook():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }

    dk_cats = {"PTS": 12488, "AST": 12495, "REB": 12492, "STL": 13508, "BLK": 13780, "STL_BLK": 13781, "TPM": 12497, 
               "PA": 9973, "PR": 9976, "RA": 9974, "PRA": 5001}
    df_lines = pd.DataFrame()
    for key, value in dk_cats.items():
        response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C{value}&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%27{value}%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%27{value}%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
        if response.status_code != 200:
            raise Exception('Bad Request')

        plyr_names = []
        pnt_lines = []
        for i in response.json()['selections']:
            plyr_names.append(i['participants'][0]['name'])
            pnt_lines.append(i['points'])
        df_dk = pd.DataFrame({"Player": plyr_names, f"{key}_line": pnt_lines}).drop_duplicates().reset_index(drop=True)

        if df_lines.empty:
            df_lines = df_dk
        else:
            df_lines = pd.merge(df_lines, df_dk, on="Player", how="outer")

    df_lines = pd.merge(pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv"), df_lines, on='Player', how='right')
    df_lines.insert(0, 'Date', pd.to_datetime(now))

    partition_save_df(df_lines, f"../tables/{year}/parlay_lines.csv") 
    display(df_lines)

    return df_lines

df_lines = get_sportsbook()

../tables/2025/parlay_lines.csv saved!


,Date,Team,Player,Pos,PTS_line,AST_line,REB_line,STL_line,BLK_line,STL_BLK_line,TPM_line,PA_line,PR_line,RA_line,PRA_line
0,2025-11-20,MIL,AJ Green,SG,9.5,2.5,2.5,NaN,NaN,NaN,2.5,12.5,13.5,5.5,15.5
1,2025-11-20,PHI,Andre Drummond,C,11.5,NaN,12.5,NaN,NaN,NaN,NaN,NaN,24.5,NaN,NaN
2,2025-11-20,ORL,Anthony Black,PG,11.5,2.5,3.5,NaN,NaN,NaN,1.5,14.5,14.5,5.5,17.5
3,2025-11-20,MIL,Bobby Portis,PF,14.5,NaN,7.5,NaN,NaN,NaN,1.5,NaN,22.5,NaN,NaN
4,2025-11-20,LAC,Bogdan Bogdanovic,SG,11.5,2.5,3.5,NaN,NaN,NaN,1.5,14.5,15.5,6.5,18.5
5,2025-11-20,LAC,Brook Lopez,C,6.5,NaN,2.5,NaN,NaN,NaN,0.5,NaN,8.5,NaN,NaN
6,2025-11-20,MEM,Cedric Coward,SG,16.5,3.5,6.5,NaN,NaN,NaN,1.5,20.5,23.5,10.5,27.5
7,2025-11-20,MIL,Cole Anthony,PG,10.5,4.5,3.5,NaN,NaN,NaN,0.5,14.5,13.5,7.5,17.5
8,2025-11-20,SAS,De'Aaron Fox,PG,24.5,7.5,3.5,1.5,NaN,NaN,2.5,32.5,28.5,11.5,36.5
9,2025-11-20,SAC,DeMar DeRozan,PF,18.5,3.5,3.5,0.5,NaN,NaN,NaN,22.5,22.5,7.5,25.5


In [59]:
pos = 'PG'
stat = 'PTS'

df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                        (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                        WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                        AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                        GROUP BY Team, Player, Pos)""").fetchdf()
df_rk_l5 = con.execute(f"""WITH last5 AS (
                                SELECT Team, Player, Pos, {stat} AS stat_val,
                                ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                FROM df
                                WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                                AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                        )
                        SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat},
                        FROM last5
                        WHERE rn <= 5
                        GROUP BY Team, Player, Pos
                        """).fetchdf()

df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                        ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()

display(df_rk)

,Team,Player,Pos,Off_PTS,Off_L5_PTS,Off_Rk
0,MIL,Giannis Antetokounmpo,PF,31.153846,27.60,1
1,ORL,Paolo Banchero,PF,21.666667,19.40,2
2,SAC,DeMar DeRozan,PF,18.666667,14.40,3
3,PHI,Kelly Oubre Jr.,PF,18.000000,15.75,4
4,MIL,Kyle Kuzma,PF,13.285714,14.80,5
...,...,...,...,...,...,...
106,ORL,Noah Penda,SF,3.400000,3.40,13
107,PHI,Kelly Oubre Jr.,SF,3.000000,15.75,14
108,MIL,Thanasis Antetokounmpo,SF,2.000000,2.00,15
109,PHI,Kelly Oubre Jr.,PF,18.000000,3.00,4


In [64]:
def pick_finder(stat, collect=False):
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])
    
    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_rk_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Team, Player, Pos, {stat} AS stat_val,
                                    ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE Season = 2025 AND {stat} > 0 AND Pos != 'None'
                                    AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                            )
                            SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat},
                            FROM last5
                            WHERE rn <= 5
                            GROUP BY Team, Player, Pos
                            """).fetchdf()

    df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                            ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})""").fetchdf()

        df_def_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Opp AS Team, {stat} AS stat_val, 
                                    ROW_NUMBER() OVER (PARTITION BY Opp ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE MP >= 27 AND Season = 2025 AND Pos = '{pos}'
                                    )

                                    SELECT Team, '{pos}' AS Pos, AVG(stat_val) AS Def_L5_{stat} FROM last5
                                    WHERE rn <= 5
                                    GROUP BY Team
                                    HAVING Team IN ({str(tms_today).replace('[','').replace(']','')})
                                    """).fetchdf()
        df_def = con.execute(f"""SELECT df_def.Team, df_def.Pos, Def_{stat}, Def_L5_{stat}, Def_Rk FROM df_def 
                               JOIN df_def_l5 ON df_def.Team = df_def_l5.Team""").fetchdf()
        
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_L5_{stat}, 
                                   df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()
        
        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date >= '2023-10-23'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_L5_{stat}, t2.AVG_{stat}_H2H, Off_Rk, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_L5_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
        df_picks = con.execute(f"""SELECT df_picks.*, df_lines.{stat}_line FROM df_picks LEFT JOIN df_lines 
                                   ON df_picks.Team = df_lines.Team AND df_picks.Player = df_lines.Player""").fetchdf()
        df_picks[f'{stat}_Diff'] = df_picks[f'Off_{stat}'] - df_picks[f'{stat}_line']
        df_picks[f'{stat}_Diff2'] = abs(df_picks[f'{stat}_Diff'])
        df_picks = df_picks[~(df_picks[f'{stat}_Diff'].isnull())].sort_values(f'{stat}_Diff2', ascending=False).reset_index(drop=True).drop(f'{stat}_Diff2', axis=1)
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [65]:
for i in categories:
    print(f"==={i}===")
    pick_finder(i)

===PTS===
PG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk,PTS_line,PTS_Diff
0,MIL,1,Ryan Rollins,17.400000,18.4,4.166667,6,PHI,18.083333,16.8,6,20.5,-3.100000
1,PHI,1,Tyrese Maxey,31.928571,29.6,31.000000,1,MIL,20.642857,22.2,15,29.5,2.428571
2,ORL,0,Jalen Suggs,12.545455,15.0,NaN,11,LAC,22.076923,25.6,21,14.5,-1.954545
3,SAS,0,De'Aaron Fox,22.666667,22.4,25.200000,3,ATL,19.727273,22.6,10,24.5,-1.833333
4,LAC,0,James Harden,26.153846,32.8,NaN,2,ORL,22.384615,19.8,22,27.5,-1.346154
5,SAC,1,Dennis Schroder,13.785714,13.8,15.800000,9,MEM,23.818182,25.8,28,12.5,1.285714
6,ORL,0,Tyus Jones,4.666667,4.8,7.833333,17,LAC,22.076923,25.6,21,3.5,1.166667
7,MIL,1,Cole Anthony,9.461538,6.8,6.857143,14,PHI,18.083333,16.8,6,10.5,-1.038462
8,SAC,1,Russell Westbrook,13.866667,11.6,12.714286,8,MEM,23.818182,25.8,28,13.5,0.366667
9,LAC,0,Kris Dunn,7.769231,7.2,5.500000,15,ORL,22.384615,19.8,22,7.5,0.269231


SG


,Team,B2B,Player,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Off_Rk,Opp,Def_PTS,Def_L5_PTS,Def_Rk,PTS_line,PTS_Diff
0,MEM,1,Vince Williams Jr.,9.555556,10.2,0.500000,13,SAC,19.608696,18.2,28,13.5,-3.944444
1,ORL,0,Desmond Bane,16.933333,22.4,19.000000,4,LAC,15.714286,13.4,8,20.5,-3.566667
2,MEM,1,Jaylen Wells,9.466667,10.4,NaN,14,SAC,19.608696,18.2,28,12.5,-3.033333
3,ATL,0,Dyson Daniels,8.500000,6.4,6.500000,17,SAS,16.363636,10.0,11,11.5,-3.000000
4,SAS,0,Devin Vassell,13.571429,11.2,18.000000,9,ATL,15.818182,20.4,9,16.5,-2.928571
5,MIL,1,Gary Trent Jr.,10.200000,7.6,14.750000,12,PHI,18.375000,20.2,23,12.5,-2.300000
6,MEM,1,Cedric Coward,14.333333,13.0,NaN,7,SAC,19.608696,18.2,28,16.5,-2.166667
7,SAC,1,Zach LaVine,21.538462,15.8,NaN,1,MEM,18.181818,16.0,21,19.5,2.038462
8,PHI,1,Quentin Grimes,17.071429,16.0,9.166667,3,MIL,18.666667,22.0,26,15.5,1.571429
9,PHI,1,VJ Edgecombe,15.857143,14.0,NaN,5,MIL,18.666667,22.0,26,14.5,1.357143


SF


...

,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Diff
0,ATL,0,Jalen Johnson,2.500000,4.0,NaN,3,SAS,1.368421,1.6,6,1.5,1.000000
1,ORL,0,Franz Wagner,1.846154,2.4,2.5,9,LAC,1.571429,1.8,10,1.5,0.346154


PF


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Diff


C


,Team,B2B,Player,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Off_Rk,Opp,Def_STL_BLK,Def_L5_STL_BLK,Def_Rk,STL_BLK_line,STL_BLK_Diff
0,ATL,0,Kristaps Porzingis,2.375000,2.6,NaN,6,SAS,1.857143,1.6,7,1.5,0.875000
1,LAC,0,Ivica Zubac,1.833333,1.8,0.75,13,ORL,1.846154,1.6,6,1.5,0.333333


In [66]:
df_stats = pd.DataFrame()
for i in categories:
    print(f"==={i}===")
    df_temp = pick_finder(i, collect=True)
    df_stats = pd.concat([df_stats, df_temp], ignore_index=True)
    
df_stats = con.execute("SELECT Date, Team, B2B, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk) FROM df_stats").fetchdf()
df_save = pd.DataFrame()
for player in df_stats.Player.unique():
    df_temp = df_stats[(df_stats.Player == player)]
    for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
        df_temp[col] = df_temp[col].ffill()
        df_temp[col] = df_temp[col].backfill()
    df_temp = df_temp.drop_duplicates()
    df_save = pd.concat([df_save, df_temp])
print('Saving for Date:', now)
partition_save_df(df_save, f"../tables/{year}/parlay_stats.csv")
display(df_save)

===PTS===
PG
SG
SF
PF
C
===AST===
PG
SG
SF
PF
C
===REB===
PG
SG
SF
PF
C
===PR===
PG
SG
SF
PF
C
===PA===
PG
SG
SF
PF
C
===RA===
PG
SG
SF
PF
C
===PRA===
PG
SG
SF
PF
C
===TPM===
PG
SG
SF
PF
C
===STL===
PG
SG
SF
PF
C
===BLK===
PG
SG
SF
PF
C
===STL_BLK===
PG
SG
SF
PF
C
Saving for Date: 2025-11-20
../tables/2025/parlay_stats.csv saved!


,Date,Team,B2B,Player,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,PTS_Diff,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,AST_Diff,Off_REB,Off_L5_REB,AVG_REB_H2H,Def_REB,Def_L5_REB,REB_line,REB_Diff,Off_PR,Off_L5_PR,AVG_PR_H2H,Def_PR,Def_L5_PR,PR_line,PR_Diff,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,PA_Diff,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,RA_Diff,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,PRA_Diff,Off_TPM,Off_L5_TPM,AVG_TPM_H2H,Def_TPM,Def_L5_TPM,TPM_line,TPM_Diff,Off_STL,Off_L5_STL,AVG_STL_H2H,Def_STL,Def_L5_STL,STL_line,STL_Diff,Off_BLK,Off_L5_BLK,AVG_BLK_H2H,Def_BLK,Def_L5_BLK,BLK_line,BLK_Diff,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,Def_L5_STL_BLK,STL_BLK_line,STL_BLK_Diff
0,2025-11-20,MIL,1,Ryan Rollins,PHI,17.400000,18.40,4.166667,18.083333,16.8,20.5,-3.100000,5.333333,5.2,1.500000,5.833333,6.0,5.5,-0.166667,4.000000,4.2,1.500000,5.500000,4.0,4.5,-0.500000,21.133333,22.60,5.666667,23.583333,20.8,24.5,-3.366667,22.733333,23.6,5.666667,23.916667,22.8,26.5,-3.766667,9.066667,9.4,3.000000,11.333333,10.0,10.5,-1.433333,26.466667,27.8,7.166667,29.416667,26.8,30.5,-4.033333,2.846154,3.600000,0.500000,2.083333,1.8,2.5,0.346154,2.083333,1.40,0.333333,1.416667,1.4,1.5,0.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-20,PHI,1,Tyrese Maxey,MIL,31.928571,29.60,31.000000,20.642857,22.2,29.5,2.428571,7.785714,6.8,5.666667,6.000000,6.0,6.5,1.285714,4.714286,4.2,4.500000,5.571429,6.8,3.5,1.214286,36.642857,33.80,35.500000,26.214286,29.0,34.5,2.142857,39.714286,36.4,36.666667,26.642857,28.2,36.5,3.214286,12.500000,11.0,10.166667,11.571429,12.8,10.5,2.000000,44.428571,40.6,41.166667,32.214286,35.0,41.5,2.928571,3.928571,3.600000,4.000000,2.000000,2.0,3.5,0.428571,1.615385,2.20,1.333333,1.428571,0.8,1.5,0.115385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-11-20,ORL,0,Jalen Suggs,LAC,12.545455,15.00,NaN,22.076923,25.6,14.5,-1.954545,4.727273,5.2,NaN,5.461538,3.4,4.5,0.227273,4.000000,5.0,NaN,4.076923,4.4,4.5,-0.500000,16.545455,20.00,NaN,26.153846,30.0,18.5,-1.954545,17.272727,20.2,NaN,27.538462,29.0,19.5,-2.227273,8.727273,10.2,NaN,9.538462,7.8,9.5,-0.772727,21.272727,25.2,NaN,31.615385,33.4,23.5,-2.227273,2.000000,2.200000,NaN,3.153846,2.6,1.5,0.500000,2.571429,2.80,NaN,1.384615,1.6,1.5,1.071429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-11-20,SAS,0,De'Aaron Fox,ATL,22.666667,22.40,25.200000,19.727273,22.6,24.5,-1.833333,6.166667,6.8,6.400000,6.818182,8.4,7.5,-1.333333,3.166667,3.2,5.000000,6.545455,7.2,3.5,-0.333333,25.833333,25.60,30.200000,26.272727,29.8,28.5,-2.666667,28.833333,29.2,31.600000,26.545455,31.0,32.5,-3.666667,9.333333,10.0,11.400000,13.363636,15.6,11.5,-2.166667,32.000000,32.4,36.600000,33.090909,38.2,36.5,-4.500000,2.500000,2.600000,3.400000,2.181818,2.8,2.5,0.000000,1.750000,1.75,2.800000,1.454545,1.2,1.5,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-11-20,LAC,0,James Harden,ORL,26.153846,32.80,NaN,22.384615,19.8,27.5,-1.346154,8.692308,8.0,NaN,7.153846,5.8,7.5,1.192308,6.833333,9.0,NaN,4.923077,5.0,5.5,1.333333,32.461538,41.80,NaN,27.307692,24.8,33.5,-1.038462,34.846154,40.8,NaN,29.538462,25.6,35.5,-0.653846,15.000000,17.0,NaN,12.076923,10.8,14.5,0.500000,41.153846,49.8,NaN,34.461538,30.6,41.5,-0.346154,3.615385,4.000000,NaN,2.230769,2.4,3.5,0.115385,1.500000,1.00,NaN,1.846154,1.2,1.5,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2025-11-20,SAC,1,Dennis Schroder,MEM,13.785714,13.80,15.800000,23.818182,25.8,12.5,1.285714,5.733333,4.2,6.200000,5.181818,6.0,4.5,1.233333,4.000000,4.6,2.700000,5.090909,5.0,3.5,0.500000,17.785714,18.40,18.500000,28.909091,30.8,15.5,2.285714,18.600000,17.4,22.000000,29.000000,31.8,17.5,1.100000,9.466667,8.2,8.900000,10.272727,11.0,8.5,0.966667,22.333333,21.4,24.700000,34.090909,36.8,20.5,1.833333,2.400000,2.800000,2.300000,3.545455,3.6,1.5,0.900000,1.363636,1.00,0.600000,0.636364,1.0,0.5,0.